<a href="https://colab.research.google.com/github/nematatu/Colab-Python/blob/master/VGG16%E7%94%BB%E5%83%8F%E5%88%86%E9%A1%9E%E3%83%86%E3%82%B9%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install -q bing_image_downloader
!pip install -q google-cloud-vision
!pip install -q tensorflow
!pip install -q Pillow
!pip -q install icrawler

In [31]:
from bing_image_downloader import downloader
from google.cloud import vision
import google.cloud.vision_v1.types as types
from PIL import Image as PILImage
from PIL import ImageOps
import os
import random
import shutil
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import glob
from scipy import ndimage
from icrawler.builtin import GoogleImageCrawler

from google.colab import drive
from IPython.display import Image,display_jpeg

In [32]:
!rm -rf ./ds ./img

In [33]:
drive.mount('/content/drive')
google_auth_path='/content/drive/MyDrive/Colab\ Notebooks/kasoon_colab/fluted-oath-407809-b8c332499f57.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] ='/content/drive/MyDrive/開発/kasoon/fluted-oath-407809-b8c332499f57.json'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
SearchNames=['伊織もえ','えなこ','田中美久','斎藤飛鳥']

for query_string in SearchNames:
  downloader.download(query_string,output_dir=r'./img',adult_filter_off=False,timeout=60,verbose= True)

ストリーミング出力は最後の 5000 行に切り捨てられました。
[%] File Downloaded !

[%] Downloading Image #16 from https://64.media.tumblr.com/4a4890339366df2add14bfa4d55788fa/61fb090ea2eb0603-f6/s1280x1920/5bcd9917b11fdc5e38dc2345b4e99ca2e3570e55.jpg
[Error]Invalid image, not saving https://64.media.tumblr.com/4a4890339366df2add14bfa4d55788fa/61fb090ea2eb0603-f6/s1280x1920/5bcd9917b11fdc5e38dc2345b4e99ca2e3570e55.jpg

[!] Issue getting: https://64.media.tumblr.com/4a4890339366df2add14bfa4d55788fa/61fb090ea2eb0603-f6/s1280x1920/5bcd9917b11fdc5e38dc2345b4e99ca2e3570e55.jpg
[!] Error:: Invalid image, not saving https://64.media.tumblr.com/4a4890339366df2add14bfa4d55788fa/61fb090ea2eb0603-f6/s1280x1920/5bcd9917b11fdc5e38dc2345b4e99ca2e3570e55.jpg

[%] Downloading Image #16 from https://pbs.twimg.com/media/Fl0xFaRaMAAFUQU.jpg
[%] File Downloaded !

[%] Downloading Image #17 from https://i.pinimg.com/736x/a6/51/b6/a651b63f8fdc65d204a3ed4c993cc5d7.jpg
[%] File Downloaded !

[%] Downloading Image #18 from https://cdn.v2p

In [35]:
#def: 顔の検知
def detect_face(image_path):
    client=vision.ImageAnnotatorClient()
    #コードブロックが終わったら自動でfileが閉じる
    #メモリの効率化
    #引数で取ったiamge_pathを'rb'バイナリファイルを読み込む
    with open(image_path, 'rb') as file:
        #読み込んだバイナリファイルを画像形式に変換している
        image=types.Image(content=file.read())
        #GCV APIのface_detection()を使って画像内の顔を検出する
        #そのうち、顔の情報を含むface_annotationsオブジェクトを返す
        annotations=client.face_detection(image=image).face_annotations

    face_boxes=[]

    for annotation in annotations:
        #annotation(顔の情報)の境界ボックス(bounding_poly)の座標情報(vertices)を取得
        #リスト内包表記で、うちx座標のみをリストに格納
        x_s=[vertex.x for vertex in annotation.bounding_poly.vertices]
        y_s=[vertex.y for vertex in annotation.bounding_poly.vertices]
        face_boxes.append((min(x_s),min(y_s),max(x_s),max(y_s)))
    return face_boxes

In [36]:
#def: データセットの分割
train_dir_path='./ds/train'
test_dir_path='./ds/test'

def split_dataset(image_dir_path):
  print("データセットの分割を開始します")
  file_list=os.listdir(image_dir_path)
  random.shuffle(file_list)

  dir_name=image_dir_path.split('/')[-1]
  train_size=int(len(file_list)*0.8)
  print(f"images:{int(len(file_list))}")
  print(f"train:{train_size}")
  if not os.path.exists('./ds'):
    os.mkdir('./ds')
    os.mkdir(train_dir_path)
    os.mkdir(test_dir_path)
  os.mkdir(os.path.join(test_dir_path,dir_name))
  os.mkdir(os.path.join(train_dir_path,dir_name))
  for i, file in enumerate(file_list):
    if i < train_size:
      shutil.copy(os.path.join(image_dir_path, file), os.path.join(train_dir_path,dir_name, file))
    else:
      shutil.copy(os.path.join(image_dir_path, file), os.path.join(test_dir_path, dir_name,file))
  print("データセットの分割が終了しました！")

In [37]:
#def: 底上げ
def false_buttom(in_dir,query_string):
  print(f"{query_string}の写真を水増しします")
  out_dir = "./img/Edited/" + query_string
  if not os.path.exists(os.path.join("./img","Edited")):
    os.mkdir(os.path.join("./img","Edited"))
  if not os.path.exists(out_dir):
    os.mkdir(out_dir)
  in_jpg = glob.glob(os.path.join(in_dir,'*'))
  for i in range(len(in_jpg)):
      img = cv2.imread(str(in_jpg[i]))
      print(str(in_jpg[i]))
      for ang in [-10, 0, 10]:
          img_rot = ndimage.rotate(img, ang)
          fileName = os.path.join(out_dir, str(i) + "_" + str(ang) + ".jpg")
          cv2.imwrite(str(fileName), img_rot)
          retval, img_thr = cv2.threshold(img_rot, 100, 255, cv2.THRESH_TOZERO)
          fileName = os.path.join(out_dir, str(i) + "_" + str(ang) + "thr.jpg")
          cv2.imwrite(str(fileName), img_thr)
          img_filter = cv2.GaussianBlur(img_rot, (5, 5), 0)
          fileName = os.path.join(out_dir, str(i) + "_" + str(ang) + "filter.jpg")
          cv2.imwrite(str(fileName), img_filter)
  print("画像の水増しに大成功しました！")

In [38]:
#body
max_margin=0.2
image_size=(128,128)
for query_string in SearchNames:
  image_dir_path=os.path.join('./img',query_string)
  file_list=os.listdir(image_dir_path)
  for img in file_list:
      new_img=os.path.join(image_dir_path,img)
      target_name=image_dir_path.split('/')[-1]
      print(new_img)

      if img.endswith('.jpg') or img.endswith('.png') or img.endswith('.jpeg'):
          #faces=img+'_face'
          #GoogleDriveのパスを空白を含めないディレクトリに変更したら行けた
          faces=detect_face(new_img)

          if len(faces)>0:
              #PILライブラリのImage.open()で画像を開く
              #from PIL import Image as PILImageとしたらいけた
              #恐らく、GoogleColab内でImageという変数が重複してる
              image=PILImage.open(new_img).convert('RGB')

              x1,y1,x2,y2=faces[0]
              w=x2-x1
              h=y2-y1

              spaces_x = min(x1, image.width - x2, int(float(w) * max_margin))
              spaces_y = min(y1,image.height-y2,int(float(h)*max_margin))
              margin=min(spaces_x,spaces_y)

              cropped_img=image.crop((
                  x1-margin,
                  y1-margin,
                  x2+margin,
                  y2+margin
              ))
              #ImageOps:切り取り
              cropped_img=ImageOps.contain(cropped_img,image_size)

              directory='face_cropped'
              new_directory=os.path.join('./img',directory,target_name)

              if not os.path.exists(os.path.join('./img',directory)):
                os.mkdir(os.path.join('./img',directory))
              if not os.path.exists(new_directory):
                  os.mkdir(new_directory)

              cropped_img.save(os.path.join(new_directory,f'cropped_{img}'))

  false_buttom(os.path.join("./img","face_cropped",query_string),query_string)
  split_dataset(os.path.join('./img','Edited',query_string))



./img/伊織もえ/Image_49.jpg
./img/伊織もえ/Image_85.jpg
./img/伊織もえ/Image_54.jpg
./img/伊織もえ/Image_77.jpg
./img/伊織もえ/Image_13.jpg
./img/伊織もえ/Image_66.jpg
./img/伊織もえ/Image_47.jpg
./img/伊織もえ/Image_97.jpg
./img/伊織もえ/Image_50.jpg
./img/伊織もえ/Image_27.jpg
./img/伊織もえ/Image_71.jpg
./img/伊織もえ/Image_92.jpg
./img/伊織もえ/Image_76.jpg
./img/伊織もえ/Image_45.jpg
./img/伊織もえ/Image_69.jpg
./img/伊織もえ/Image_67.JPG
./img/伊織もえ/Image_73.jpg
./img/伊織もえ/Image_34.jpg
./img/伊織もえ/Image_21.jpg
./img/伊織もえ/Image_30.jpg
./img/伊織もえ/Image_59.jpg
./img/伊織もえ/Image_87.jpg
./img/伊織もえ/Image_14.jpg
./img/伊織もえ/Image_42.jpg
./img/伊織もえ/Image_93.JPG
./img/伊織もえ/Image_91.jpg
./img/伊織もえ/Image_1.jpg
./img/伊織もえ/Image_74.jpg
./img/伊織もえ/Image_72.jpg
./img/伊織もえ/Image_70.jpg
./img/伊織もえ/Image_38.jpg
./img/伊織もえ/Image_32.jpg
./img/伊織もえ/Image_89.jpg
./img/伊織もえ/Image_12.jpg
./img/伊織もえ/Image_3.jpg
./img/伊織もえ/Image_75.jpg
./img/伊織もえ/Image_26.jpg
./img/伊織もえ/Image_61.jpg
./img/伊織もえ/Image_40.jpg
./img/伊織もえ/Image_98.jpg
./img/伊織もえ/Image_19.jpg
./img/伊織もえ/Image_2